In [1]:
import requests
import json

In [2]:
def get_award_data(keyword, exp_date_end=None, start_offset=1, limit=25, num_batches=10, output_file="awards.json"):
    base_url = "http://api.nsf.gov/services/v1/awards.json"
    all_fields = (
        "rpp,offset,id,agency,awardeeCity,awardeeCountryCode,awardeeDistrictCode,awardeeName,"
        "awardeeStateCode,awardeeZipCode,cfdaNumber,coPDPI,date,startDate,expDate,estimatedTotalAmt,"
        "fundsObligatedAmt,ueiNumber,fundProgramName,parentUeiNumber,pdPIName,perfCity,perfCountryCode,"
        "perfDistrictCode,perfLocation,perfStateCode,perfZipCode,poName,primaryProgram,transType,title,"
        "awardee,poPhone,poEmail,awardeeAddress,perfAddress,publicationResearch,publicationConference,"
        "fundAgencyCode,awardAgencyCode,projectOutComesReport,abstractText,piFirstName,piMiddleInitial,piLastName,piEmail"
    )
    results = []
    
    for batch in range(num_batches):
        offset = start_offset + batch * limit
        params = {
            "callback": "processJson",
            "keyword": keyword,
            "rpp": limit,
            "offset": offset,
            "printFields": all_fields
        }
        
        if exp_date_end:
            params["expDateEnd"] = exp_date_end
        
        response = requests.get(base_url, params=params)
        if response.status_code == 200:
            # Extract JSON data from response text (remove 'processJson(' prefix and ')' suffix)
            json_text = response.text[len("processJson("):-1]
            try:
                data = json.loads(json_text)
                if 'response' in data and 'award' in data['response']:
                    results.extend(data['response']['award'])
                elif 'award' in data:
                    results.extend(data['award'])
                else:
                    print(f"Unexpected response structure in batch {batch + 1}: {data}")
            except json.JSONDecodeError as e:
                print(f"JSON decoding error in batch {batch + 1}: {e}")
        else:
            print(f"Failed to fetch batch {batch + 1}: {response.status_code}")
            break

    # Filter results to include only those with a project outcomes report
    results_with_outcomes = [award for award in results if 'projectOutComesReport' in award]

    # Save results to a file
    with open(output_file, "w") as f:
        json.dump(results_with_outcomes, f, indent=4)
        
    print(f"Data saved to {output_file}")

In [3]:
keyword = "animal sounds language"
exp_date_end = "07/30/2024"
# exp_date_end = None
# start_offset = 251
start_offset = 1
output_file = "awards_new.json"
get_award_data(keyword, exp_date_end=exp_date_end, start_offset=start_offset, num_batches=5, output_file=output_file)

Data saved to awards_new.json
